In [1]:

import h5py
import pandas as pd
import numpy as np
#from plotnine import *
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import cohen_kappa_score
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.ensemble import GradientBoostingClassifier
    
#import statsmodels.api as sm

#from scipy.stats import mode

import random
from IPython.display import Image
import pydot
from pydot import Dot
 
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import seaborn as sns
sns.set()

import tensorflow as tf
#import keras-gpu
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical
#from keras.np_utils import probas_to_classes 
from keras.layers import Dense, Dropout, Input, Reshape
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D, merge, Concatenate, Conv2D, MaxPooling2D
from keras.layers.merge import concatenate, add
from keras.models import load_model
from keras.utils import plot_model
from keras.callbacks import TensorBoard
from keras import regularizers

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
import random as rn
import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["PATH"] += os.pathsep + 'C:\\Program Files (x86)\\Graphviz2.38\\bin\\'
np.random.seed(42)
rn.seed(12345)

Using TensorFlow backend.


Ask tenserflow not to be too gready on GPU

In [2]:
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
#config = tf.ConfigProto()
session_conf.gpu_options.allow_growth = True
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)

K.set_session(sess)

keras.__version__

#with eeg only 5 filters, kernel size 100
#Test score:  1.3631985359422534
#Test accuracy:  0.6440793987471519

'2.2.0'

In [3]:
# %load "C:\\Users\\i053131\Desktop\\Epilepsie\\Dreem\\src\\utils\\error.py"

def AnalyzeError(y_true, y_pred):
    fig, ax = plt.subplots(figsize=(20,10))
    plt.subplot(1,2, 1)
    sns.countplot(x=0, data=pd.DataFrame(y_true))
    plt.ylim(0, 4000)
    plt.subplot(1,2, 2)
    sns.countplot(x=0, data=pd.DataFrame(y_pred))
    plt.ylim(0, 4000)
    fig.suptitle("Actual and predicted distribution", size =  'x-large')
    plt.show()
    
    df_ = pd.DataFrame()
    df_["Test"]= y_true
    df_["Pred"] = y_pred
    df_['error'] = df_.Test != df_.Pred
    #sns.countplot(x="Test", data=df_[df_.error])
    
    error0 = df_[(df_.error) & (df_.Test==0)].count()[0] / df_[df_.Test==0].count()[0]
    error1 = df_[(df_.error) & (df_.Test==1)].count()[0] / df_[df_.Test==1].count()[0]
    error2 = df_[(df_.error) & (df_.Test==2)].count()[0] / df_[df_.Test==2].count()[0]
    error3 = df_[(df_.error) & (df_.Test==3)].count()[0] / df_[df_.Test==3].count()[0]
    error4 = df_[(df_.error) & (df_.Test==4)].count()[0] / df_[df_.Test==4].count()[0]

    Lerror = [error0, error1, error2, error3, error4]
    sns.barplot(x=[0, 1, 2, 3, 4], y=Lerror)
    plt.title('Wrongly classified in a phase in percent of the test population for this phase')
    plt.show()

In [4]:
#trying with hyperas
# downgrade a package :conda install networkx==1.11

def data():
    dataPath = "C:\\Users\\i053131\\Desktop\\Epilepsie\\Dreem\\data\\raw\\"

    trainOutput = pd.read_csv(dataPath + "challenge_fichier_de_sortie_dentrainement_classification_en_stade_de_sommeil_a_laide_de_signaux_mesures_par_le_bandeau_dreem.csv", sep=";")
    Y = trainOutput["label"]

    filetrain= dataPath + "train.h5"
    filetest= dataPath + "test.h5"
    h5 = h5py.File(filetrain, "r")

    eeg_1 = pd.DataFrame(h5['eeg_1'][:])
    eeg_2 = pd.DataFrame(h5['eeg_2'][:])
    eeg_3 = pd.DataFrame(h5['eeg_3'][:])
    eeg_4 = pd.DataFrame(h5['eeg_4'][:])
    po_ir = pd.DataFrame(h5['po_ir'][:])
    po_r = pd.DataFrame(h5['po_r'][:])
    accelerometer_x = pd.DataFrame(h5['accelerometer_x'][:])
    accelerometer_y = pd.DataFrame(h5['accelerometer_y'][:])
    accelerometer_z = pd.DataFrame(h5['accelerometer_z'][:])

    df = pd.concat([eeg_1, eeg_2, eeg_3, eeg_4, po_ir, po_r, accelerometer_x, accelerometer_y, accelerometer_z], 
                   axis=1, sort = False)
    df.columns = range(15000 + 3000 + 4500)
    df["Y"] = Y

    training, test  = train_test_split(df, test_size=0.2, random_state=42)
    X = training.iloc[:,:-1]
    X_train = X.values
    y = training.iloc[:,-1]
    y_train = to_categorical(y.values, num_classes=5)
    X_test = test.iloc[:,:-1].values
    y_true = test.iloc[:,-1].values
    y_test = to_categorical(y_true, num_classes=5)

    eeg_train = np.empty(shape=(35064, 3750, 4))
    eeg_train[:, :, 0]= X_train[:, :3750] #eeg_1
    eeg_train[:, :, 1]= X_train[:, 3750:3750*2] #eeg_2
    eeg_train[:, :, 2]= X_train[:, 3750*2:3750*3] #eeg_3
    eeg_train[:, :, 3]= X_train[:, 3750*3:3750*4] #eeg_4

    eeg_test = np.empty(shape=(8766, 3750, 4))
    eeg_test[:, :, 0]= X_test[:, :3750] #eeg_1
    eeg_test[:, :, 1]= X_test[:, 3750:3750*2] #eeg_2
    eeg_test[:, :, 2]= X_test[:, 3750*2:3750*3] #eeg_3
    eeg_test[:, :, 3]= X_test[:, 3750*3:3750*4] #eeg_4

    print("eeg_train", eeg_train.shape)

    pulse_train = np.empty(shape=(35064, 1500, 2))
    pulse_train[:, :, 0]= X_train[:, 3750*4 : 3750*4 + 1500] #po_ir
    pulse_train[:, :, 1]= X_train[:, 3750*4 + 1500 : 3750*4 +1500*2] #po_r

    pulse_test = np.empty(shape=(8766, 1500, 2))
    pulse_test[:, :, 0]= X_test[:, 3750*4 : 3750*4 + 1500] #po_ir
    pulse_test[:, :, 1]= X_test[:, 3750*4 + 1500 : 3750*4 + 1500*2] #po_r

    print("pulse_train", pulse_train.shape)

    accelerometer_train = np.empty(shape=(35064, 1500, 3))
    accelerometer_train[:, :, 0]= X_train[:, 3750*4 + 1500*2 : 3750*4 + 1500*3] #accelerometer_x
    accelerometer_train[:, :, 1]= X_train[:, 3750*4 + 1500*3 : 3750*4 + 1500*4] #accelerometer_y
    accelerometer_train[:, :, 2]= X_train[:, 3750*4 + 1500*4 : 3750*4 + 1500*5] #accelerometer_z

    accelerometer_test = np.empty(shape=(8766, 1500, 3))
    accelerometer_test[:, :, 0]= X_test[:, 3750*4 + 1500*2 : 3750*4 + 1500*3] #accelerometer_x
    accelerometer_test[:, :, 1]= X_test[:, 3750*4 + 1500*3 : 3750*4 + 1500*4] #accelerometer_y
    accelerometer_test[:, :, 2]= X_test[:, 3750*4 + 1500*4 : 3750*4 + 1500*5] #accelerometer_z

    print("accelerometer_train", pulse_train.shape)

    x_train = [eeg_train, pulse_train, accelerometer_train] #check
    x_test = [eeg_test, pulse_test, accelerometer_test] 
    return x_train, y_train, x_test, y_test

In [5]:
def create_model(x_train, y_train, x_test, y_test):
    ksize1=250; ksize2=100; ksize3=100
    nb_filter1={{choice([10, 20, 50])}}
    nb_filter2={{choice([50, 100, 200])}}
    dsize={{choice([250, 300, 350])}}
    reg=10; epochs=100
    
    def submodel_2D(X_train, nb_filter1=5, nb_filter2=20, kernel1=25,  maxPool1=25):
        inputs = Input(shape=X_train.shape[1:3])
        conv1 = Conv1D(nb_filter=nb_filter1, kernel_size=kernel1, activation='relu', strides = 1, padding='valid')(inputs)
        pool1 = MaxPooling1D(maxPool1)(conv1)
        return Model(inputs=inputs, outputs=pool1)
    
    #config = tf.ConfigProto()
    #config.gpu_options.allow_growth = True
    #sess = tf.Session(config=config)
    #K.set_session(sess)
    
    model_eeg = submodel_2D(x_train[0], nb_filter1=nb_filter1, kernel1=ksize1, maxPool1=20)
    model_pulse = submodel_2D(x_train[1], nb_filter1=nb_filter1,kernel1=ksize2, maxPool1=8)
    model_acc = submodel_2D(x_train[2], nb_filter1=nb_filter1, kernel1=ksize3, maxPool1=8)

    in_eeg =  Input(shape=x_train[0].shape[1:3])
    in_pulse = Input(shape=x_train[1].shape[1:3])
    in_acc = Input(shape=x_train[2].shape[1:3])
    out_eeg = model_eeg(in_eeg)
    out_pulse = model_pulse(in_pulse)
    out_acc = model_acc(in_acc)
    merged = concatenate([out_eeg, out_pulse, out_acc], axis=-1) 
    steps = int(merged.shape[1])
    filters = int(merged.shape[2])
    stacked_shape = (None, steps, filters, 1)
    stacked = Reshape((steps, filters, 1))(merged)
    conv3 = Conv2D(nb_filter=nb_filter2, kernel_size=(40, 30), activation='relu', strides=(1, 1), padding='valid', 
                   data_format="channels_last")(stacked)
    pool3 = MaxPooling2D(pool_size=(20, 1))(conv3)

    flat = Flatten()(pool3)
    dense1 = Dense(dsize, activation='relu')(flat)
    #drop1 = Dropout(0.5)(dense1)
    dense2 = Dense(dsize, activation='relu')(dense1)
    out = Dense(5, activation='softmax', kernel_regularizer=regularizers.l2(reg))(dense2)
    model = Model([in_eeg, in_pulse, in_acc], out)
    optimizer=Adam(0.0001)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    model.fit(x_train, y_train, batch_size=100, epochs= epochs, 
                    validation_data = (x_test, y_test))
    score = model.evaluate(x_test, y_test, batch_size=100)
    
    #y_probas = model.predict(x_test, batch_size=100)
    #y_classes = y_probas.argmax(axis=-1) #keras.np_utils.probas_to_classes(y_probas)
    #y_pred = pd.DataFrame(y_classes)
    loss = score[0]
    accuracy = score[1]
    #kappa = cohen_kappa_score(y_true, y_pred)
    #arguments = "Ksize1: {}, Ksize2: {}, Ksize3: {}, nb_filter1: {}, nb_filter2: {}, densitysize: {}, l2: {}, epochs: {}".format(ksize1, ksize2, ksize3, nb_filter1, nb_filter2, dsize, reg, epochs)

    #model = None
    #model_eeg = None
    #model_loss = None
    #model_acc = None
    
    #return [arguments, loss, accuracy, kappa]
    del model
    gc.collect()
    #if K.backend() == 'tensorflow':
    K.clear_session()
    
    #return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}
    return {'loss': -accuracy, 'status': STATUS_OK}
    
#y_true = test.iloc[:,-1].values
#y_test = to_categorical(y_true, num_classes=5)

In [ ]:
best_run, best_model = optim.minimize(model=create_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=5,
                                      trials=Trials(), 
                                      notebook_name='CNN_hyperas')

X_train, Y_train, X_test, Y_test = data()
#print("Evalutation of best performing model:")
#print(best_model.evaluate(X_test, Y_test))
print("Best performing model chosen hyper-parameters:")
print(best_run)

>>> Imports:
#coding=utf-8

try:
    import h5py
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from sklearn.decomposition import PCA
except:
    pass

try:
    from sklearn.cluster import KMeans
except:
    pass

try:
    from sklearn.metrics import accuracy_score
except:
    pass

try:
    from sklearn.linear_model import LogisticRegression
except:
    pass

try:
    from sklearn.metrics import confusion_matrix
except:
    pass

try:
    from sklearn.metrics import precision_score, recall_score
except:
    pass

try:
    from sklearn.metrics import cohen_kappa_score
except:
    pass

try:
    from sklearn import preprocessing
except:
    pass

try:
    from sklearn.preprocessing import StandardScaler
except:
    pass

try:
    from sklearn.preprocessing import PolynomialFeatures
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.svm import Line

eeg_train (35064, 3750, 4)
pulse_train (35064, 1500, 2)
accelerometer_train (35064, 1500, 2)


C:\Users\i053131\Desktop\Epilepsie\Dreem\notebooks\temp_model.py:338: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(kernel_size=250, activation="relu", strides=1, padding="valid", filters=20)`
C:\Users\i053131\Desktop\Epilepsie\Dreem\notebooks\temp_model.py:338: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(kernel_size=100, activation="relu", strides=1, padding="valid", filters=20)`
C:\Users\i053131\Desktop\Epilepsie\Dreem\notebooks\temp_model.py:338: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(kernel_size=100, activation="relu", strides=1, padding="valid", filters=20)`
C:\Users\i053131\Desktop\Epilepsie\Dreem\notebooks\temp_model.py:363: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(40, 30), activation="relu", strides=(1, 1), padding="valid", data_format="channels_last", filters=200)`


Train on 35064 samples, validate on 8766 samples
Epoch 1/100
